# Prepare_brfss_data_for_hackathon

Prepare package to be shared on Github:

Please note the complete packages is XXXX GB. Please download **before** hackathon on USB.

Folder BRFSS:
1. brfss_level_1.csv : size = (10000 rows, 16 cols) decoded
2. brfss_level_2.csv : size = (10000 rows, 48 cols) decoded 
3. LLCP2019.csv      : size = (418268 rows, 342 cols) **coded** 
4. brfss2019_sample_raw.csv : size = (1000 rows, 15 cols) **coded**
4. explore_brfss_getting_started.ipynb : Notebook to get started with analysing #1 and #2 
5. BRFSS2019_decodingtabs_simplified.xlsx : contains decoding tables for each interview question (only required when analyzing LLCP2019.csv)
6. colnames.xlsx : contains decoding table for column names (only required when analyzing LLCP2019.csv)

For more information on 2019 BRFSS Survey Data and Documentation, see:
* https://www.cdc.gov/brfss/annual_data/2019/pdf/codebook19_llcp-v2-508.HTML
* https://www.cdc.gov/brfss/annual_data/annual_2019.html

Folder CBS
1. This is a set 
2. other set
3. an another again



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os.path import join

plt.style.use('ggplot')

# 1. Settings

In [2]:
 
data_dir = '/home/pieter/projects/brfss/data'  # Linux

SAMPLE_SEED = 342367
SAMPLE_SIZE = 10000


# Data inladen

In [3]:
# BFRSS enquete data
brfss_raw = pd.read_csv(join(data_dir, 'LLCP2019.csv'), sep='|')

# Decoderingstabellen
decoderingstabellen = pd.read_excel(join(data_dir, 'BRFSS2019_decodingtabs_simplified.xlsx'), sheet_name=None)

# Beschrijving kolomnamen
col_description = pd.read_excel(join(data_dir, 'colnames.xlsx')) 

In [4]:
brfss_raw.shape


(418268, 342)

In [5]:
# Random sample
brfss = brfss_raw.sample(n=SAMPLE_SIZE, random_state=SAMPLE_SEED)


In [6]:
# Decode column names
mapping_colnames = dict(zip(col_description.variable_name, col_description.label))

In [7]:
# Select columns

# selected_cols = col_description[col_description.include_level_2 == 1]['variable_name'].to_list()
selected_cols = col_description[col_description.include_level_1 == 1]['variable_name'].to_list()

ddf = brfss[selected_cols].apply(pd.to_numeric, downcast='integer') 

In [8]:
ddf.shape

(10000, 15)

In [9]:
decoding_cell_mapper = dict()

for col in ddf.columns:
    value = pd.to_numeric(decoderingstabellen[col]['Value'], downcast='integer', errors='ignore')
    label = decoderingstabellen[col]['Value Label']
    decoding_cell_mapper[col] = dict(zip(value, label))

In [10]:
mapping_colnames_2 = {'Reported_Weight_in_Pounds' : 'Reported_Weight_in_Kilograms',
                      'Reported_Height_in_Feet_and_Inches' : 'Reported_Height_in_Meters',
                      'How_Many_Times_Walking,_Running,_Jogging,_or_Swimming_NR1': 'How_Many_Times_per_Month_Walking_Running_etc',
                      'Minutes_or_Hours_Walking,_Running,_Jogging,_or_Swimming_NR1': 'Hours_Walking,_Running,_Jogging,_or_Swimming',
                      'How_many_times_did_you_eat_fruit?' : 'How_many_times_did_you_eat_fruit_per_month?',
                      'How_many_times_did_you_eat_dark_green_vegetables?' : 'How_many_times_did_you_eat_dark_green_vegetables_per_week?' }

In [11]:
### Finally, we got a human readible showcase dataset!
data = ddf.replace(decoding_cell_mapper).rename(columns=mapping_colnames).rename(columns=mapping_colnames_2)

In [ ]:
# Review each data column has a rather nice distribution

for col in data.columns:
    print('\n*** {} ***'.format(col))
    print(data[col].value_counts())

In [12]:
data['ID'] = data.index
data.sort_index(inplace=True)
data.to_csv('/home/pieter/projects/brfss/data/brfss_level_1.csv', sep='|', index=False)

In [16]:
df2 = pd.read_csv('/home/pieter/projects/brfss/data/brfss_level_2.csv', sep='|')

In [14]:
df.shape

(10000, 16)

In [15]:
df

,Sex_of_Respondent,General_Health,Have_any_health_care_coverage,Could_Not_See_Doctor_Because_of_Cost,Length_of_time_since_last_routine_checkup,Ever_Told_Blood_Pressure_High,Marital_Status,Education_Level,Own_or_Rent_Home,Employment_Status,Reported_Weight_in_Kilograms,Smoked_at_Least_100_Cigarettes,Frequency_of_Days_Now_Smoking,Days_in_past_30_had_alcoholic_beverage,How_many_times_did_you_eat_fruit_per_month?,ID
0,M,Very good,Yes,No,"[0, 1) years",Not asked or Missing,Divorced,6 College graduate,Group home,Retired,113.4,Yes,Every day,25.0,30.0,6
1,F,Very good,Yes,No,"[0, 1) years",Yes,Widowed,5 Some college or technical school,Group home,A homemaker,72.57,No,Not asked or Missing,0,30.0,166
2,F,Good,No,No,"[1, 2) years",Not asked or Missing,Married,4 High school graduate,Group home,A homemaker,Refused,No,Not asked or Missing,0,Never,271
3,M,Good,Yes,No,"[1, 2) years",Borderline high,Married,4 High school graduate,Rent,Retired,127.01,Yes,Not at all,0,56.0,296
4,F,Excellent,Yes,No,"[0, 1) years",Not asked or Missing,Married,6 College graduate,Group home,Employed for wages,68.04,Yes,Not at all,5.0,60.0,373
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,Excellent,Yes,No,"[0, 1) years",Not asked or Missing,Never married,6 College graduate,Group home,Employed for wages,72.57,No,Not asked or Missing,2.0,Never,417904
9996,F,Good,Yes,Yes,"[0, 1) years",Yes,Never married,6 College graduate,Rent,A homemaker,98.88,Yes,Not at all,0,60.0,417984
9997,F,Good,Yes,No,"[0, 1) years",Yes,Married,4 High school graduate,Group home,Retired,77.11,No,Not asked or Missing,0,7.0,417987
9998,M,Fair,Yes,No,"[0, 1) years",Yes,Married,2 Elementary,Group home,Retired,63.5,Don’t know/Not Sure,Not asked or Missing,0,30.0,418229


In [17]:
df2.head()

,State_FIPS_Code,Sex_of_Respondent,General_Health,Have_any_health_care_coverage,Multiple_Health_Care_Professionals,Could_Not_See_Doctor_Because_of_Cost,Length_of_time_since_last_routine_checkup,Ever_Told_Blood_Pressure_High,Currently_Taking_Blood_Pressure_Medication,How_Long_since_Cholesterol_Checked,...,"Hours_Walking,_Running,_Jogging,_or_Swimming",How_many_times_did_you_eat_fruit_per_month?,How_many_times_did_you_eat_dark_green_vegetables_per_week?,Adult_flu_shot/spray_past_12_mos,Computed_Race-Ethnicity_grouping,Calculated_sex_variable,Reported_age_in_five-year_age_categories_calculated_variable,Computed_Smoking_Status,Physical_Activity_Index,ID
0,Alabama,M,Very good,Yes,"Yes, only one",No,"[0, 1) years",Not asked or Missing,Not asked,"[0, 1) year",...,0.75,30.0,Never,Yes,White,Male,Age 70 to 74,Current Frequent Smoker,Meet Aerobic Recommendations,6
1,Alabama,F,Very good,Yes,"Yes, only one",No,"[0, 1) years",Yes,Yes,"[0, 1) year",...,Not asked or Missing,30.0,7.0,Yes,White,Female,Age 75 to 79,Never Smoked,Did Not Meet Aerobic Recommendations,166
2,Alabama,F,Good,No,No,No,"[1, 2) years",Not asked or Missing,Not asked,"[1, 2) year",...,2.0,Never,3.75,Not asked or Missing,White,Female,Age 55 to 59,Never Smoked,Meet Aerobic Recommendations,271
3,Alabama,M,Good,Yes,"Yes, only one",No,"[1, 2) years",Borderline high,Not asked,"[1, 2) year",...,Not asked or Missing,56.0,3.0,Not asked or Missing,White,Male,Age 70 to 74,Former Smoker,Did Not Meet Aerobic Recommendations,296
4,Alabama,F,Excellent,Yes,More than one,No,"[0, 1) years",Not asked or Missing,Not asked,"[0, 1) year",...,2.5,60.0,3.0,Not asked or Missing,Black,Female,Age 45 to 49,Former Smoker,Meet Aerobic Recommendations,373


In [18]:
df2.Reported_Weight_in_Kilograms

0         113.4
1         72.57
2       Refused
3        127.01
4         68.04
         ...   
9995      72.57
9996      98.88
9997      77.11
9998       63.5
9999      81.65
Name: Reported_Weight_in_Kilograms, Length: 10000, dtype: object